# Spark Session and Config

In [ ]:
df = spark.createDataFrame([("hello",), ("world",)], ["word"])
df.show()

# Loading env file that contain connection string

In [ ]:
from dotenv import load_dotenv
import os

# Load variables from .env into os.environ
load_dotenv("env")

In [ ]:
server_name = os.environ.get("SERVERNAME")
database_name = os.environ.get("DATABASENAME")
username = os.environ.get("USERNAME")
password = os.environ.get("PASSWORD")

# Standard JDBC URL format for SQL Server
jdbc_url = f"jdbc:sqlserver://{server_name}:1433;databaseName={database_name}"

In [ ]:
#table name to read
table_name = "dbo.AP18_CUSTOMER_MASTER"

# Read from our Azure Database

In [ ]:
try:
    jdbcDF = spark.read \
        .format("jdbc") \
        .option("driver", "com.microsoft.sqlserver.jdbc.SQLServerDriver") \
        .option("url", jdbc_url) \
        .option("dbtable", table_name) \
        .option("user", username) \
        .option("password", password) \
        .load()

    # Display schema and a few rows if successful
    jdbcDF.printSchema()
    jdbcDF.show(5)

except Exception as e:
    print("JDBC read failed!")
    print("Error:", e)


In [ ]:
spark.stop()

-----------------------------------------------------------

## 🧠 1. Was ist Spark? Warum PySpark?

> "Alle reden über Spark — aber was löst es eigentlich?"

- **Apache Spark** ist eine verteilte Datenverarbeitungs-Engine.
  - *Übersetzt*: Anstatt dass ein Computer alles alleine macht (wie dein Laptop mit Python), verteilt Spark die Arbeit auf *viele* Rechner oder CPU-Kerne.
  - *Einsatzgebiet*: Riesige Datenmengen und Machine Learning auf Skalenniveau.

- **Warum nicht einfach Pandas oder reines Python?**
  - *Kleine Daten passen in den Arbeitsspeicher → Pandas reicht locker.*
  - *Daten größer als RAM → Pandas stirbt.*
  - *Man braucht echte Parallelisierung → Spark.*

- **Warum PySpark?**
  - Spark ist in Scala und Java geschrieben — aber wer will für normale Datenverarbeitung bitte in Java rumfrickeln?
  - **PySpark** erlaubt dir, Spark mit Python zu steuern. Fast alle Vorteile, aber einfacher zu schreiben.

**Ehrliche Einschätzung**:  
> Nutze Spark nur, wenn du *musst* (Daten zu groß, Performance wird kritisch).  
> Ansonsten: **Pandas > PySpark** bei kleinen bis mittleren Daten.

## ⚙️ 2. Cluster Mode vs Local Mode

Stell dir vor, du hast eine schwere Datenverarbeitung.

| Modus | Was passiert | Wann verwenden | Wichtigster Punkt |
|:----|:------------|:----------------|:-----------|
| **Local Mode** | Spark läuft auf deinem Rechner, nutzt mehrere CPU-Kerne. | Entwicklung, Tests, kleine Datenmengen. | *Trainingsmodus.* |
| **Cluster Mode** | Spark läuft auf mehreren Servern (Nodes). | Produktion, Big Data. | *Echte Skalierung.* |

**Skizze:**

```plaintext
Local Mode:
+-----------------+
| Laptop          |
| [Core1][Core2]  |
| [Core3][Core4]  |
+-----------------+

Cluster Mode:
+--------------------+     +-------------------+     +-------------------+
| Worker Node 1      |     | Worker Node 2      |     | Worker Node 3      |
| [Task1][Task2]     |     | [Task3][Task4]     |     | [Task5][Task6]     |
+--------------------+     +-------------------+     +-------------------+
          \                    |                    /
           \                   |                   /
                +----------------------------------+
                |        Spark Driver Program     |
                +----------------------------------+
```

> **Wichtig**: *Im Cluster Mode steuert ein Programm viele Maschinen.*  
> **Local Mode simuliert nur ein Mini-Cluster auf deinem Laptop.*

## ⚙️ 3. Spark Architecture

In [ ]:
from IPython.display import Image

Image(url="https://spark.apache.org/docs/latest/img/cluster-overview.png")

### Spark Architektur: Cluster Overview)

Dieses Diagramm zeigt **wie Spark verteilt arbeitet**:

| Komponente | Beschreibung |
|:---|:---|
| **Driver Program** | Dein Hauptprogramm. Es steuert alles: Job-Aufteilung, Kommunikation, Fehlerbehandlung. |
| **SparkContext** | Die zentrale Verbindung vom Driver zu Spark. Du programmierst damit die Aktionen und Transformationen. |
| **Cluster Manager** | Verwaltet die Ressourcen im Cluster (CPU, RAM). Beispiele: YARN, Kubernetes, Standalone. |
| **Worker Nodes** | Die Maschinen, die die eigentliche Datenverarbeitung übernehmen. |
| **Executor** | Ein Prozess auf einem Worker, der Tasks ausführt und Daten im Speicher hält. |
| **Task** | Die kleinste Recheneinheit. Ein Spark-Job wird in viele Tasks aufgeteilt. |
| **Cache** | Zwischenspeicher (RAM) für Daten, die mehrfach gebraucht werden, damit sie nicht immer neu berechnet werden müssen. |

---

### 🔥 Wichtig zu verstehen:

- **Driver** → Teilt Jobs auf und schickt sie über den **Cluster Manager** an die **Worker Nodes**.
- Jeder **Worker** hat mindestens einen **Executor**, der wiederum mehrere **Tasks** parallel ausführt.
- **Caches** sparen Zeit, indem sie Daten im RAM statt auf der Platte halten.

---

### 🧠 Skeptische Beobachtungen:

- **Single Point of Failure**: Wenn der Driver abstürzt, ist der ganze Job verloren (außer du hast High Availability konfiguriert).
- **Ressourcenverschwendung möglich**: Executors brauchen RAM – schlecht abgestimmt → viele Out-of-Memory-Fehler.
- **Netzwerk Bottleneck**: Schweres Shuffling (viel Datentausch zwischen Workern) kann Spark-Jonehmer nicht einfach abschalten.)

# ✨ 4.`SparkSession` als Einstiegspunkt

- In Spark 2.0 und neuer ist **`SparkSession`** der *offizielle Einstiegspunkt* für jede Spark-Anwendung.
- Früher musste man `SparkContext`, `SQLContext`, `HiveContext` usw. separat erstellen — heute bündelt `SparkSession` alles in einem Objekt.

**Merksatz**:  
> **Ohne SparkSession → kein Zugriff auf Spark.**

**Beispiel:**

In [15]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("Localspark") \
    .master("local[*]") \
    .getOrCreate()
spark.sparkContext.setLogLevel("ERROR")

In [2]:
type(spark)

pyspark.sql.session.SparkSession

In [3]:
type(spark.sparkContext)

pyspark.context.SparkContext

## 🆚 5. DataFrame vs RDD (nur die Basics)

| Feature | DataFrame | RDD |
|:--------|:----------|:----|
| **Definition** | Tabelle mit Spalten und Datentypen (ähnlich SQL oder Pandas) | Rohes, unstrukturiertes Datenset (verteilte Liste von Objekten) |
| **Benutzerfreundlichkeit** | Hoch – SQL-ähnliche Operationen | Niedrig – eigene Map/Reduce-Logik schreiben |
| **Performance** | Optimiert durch Catalyst & Tungsten Engine | Weniger optimiert (du musst dich selbst um Performance kümmern) |
| **Anwendungsfall** | Klassische Datenverarbeitung, Analytics, Machine Learning Pipelines | Low-Level Transformationen, wenn extreme Flexibilität gebraucht wird |

**Skeptische Wahrheit**:  
> Wer heute noch direkt mit RDDs arbeitet, hat meist ein sehr spezielles Problem — oder kein Vertrauen in Spark-Optimierungen.

## 🔄 6. Lebenszyklus eines DataFrames in Spark

**Was passiert intern?**

1. **Definition**:  
   Du schreibst Transformationen (`select`, `filter`, `join`), aber Spark führt *noch nichts* aus.

2. **Logischer Plan**:  
   Spark erstellt einen logischen Ablaufplan (nur eine Beschreibung, noch keine Ausführung).

3. **Physikalischer Plan**:  
   Spark optimiert den logischen Plan zu einem ausführbaren Programm (z. B. Broadcast Joins, Predicate Pushdown).

4. **Ausführung (Action!)**:  
   Erst wenn eine **Action** kommt (`show()`, `collect()`, `write()`) wird der Plan tatsächlich ausgeführt → *Lazy Execution Prinzip.*

**Grafisch:**

```plaintext
Transformationen -> Logischer Plan -> Optimierter physikalischer Plan -> Task-Ausführung
```


## 📊 7. Daten erkunden und transformieren

### CSV Lesen

In [17]:
import requests

# Korrekte RAW-URL
url = "https://raw.githubusercontent.com/gadanes/spark_kurs/main/notebooks/data.csv"

# Lade die CSV-Datei herunter
response = requests.get(url)
with open("/tmp/data.csv", "wb") as f:
    f.write(response.content)

# Lese die CSV-Datei mit Spark ein
df = spark.read.option("header", "true").csv("/tmp/data.csv")
df.show()

+-------+---+----------+--------------------+-------------------+------+
|   name|age|      city|               email|          job_title|salary|
+-------+---+----------+--------------------+-------------------+------+
|   Anna| 28|    Berlin|anna.mueller@exam...|       Data Analyst| 52000|
|    Ben| 35|   Hamburg|ben.schmidt@examp...|  Software Engineer| 74000|
|  Clara| 22|    Munich|clara.klein@examp...|Marketing Assistant| 42000|
|  David| 40|   Cologne|david.schneider@e...|    Project Manager| 83000|
|    Eva| 31| Stuttgart|eva.huber@example...|     Data Scientist| 68000|
|  Felix| 29|    Berlin|felix.wagner@exam...|    DevOps Engineer|  NULL|
|   Gina| 45|   Hamburg|gina.fischer@exam...|         HR Manager| 69000|
| Hannah| 26| Frankfurt|hannah.koch@examp...|        UI Designer| 54000|
|   Igor| 38|    Munich|igor.keller@examp...|      Sales Manager| 75000|
|  Julia| 24|Düsseldorf|julia.schmitt@exa...|    Content Creator| 41000|
|   Karl| 50|    Berlin|karl.bauer@exampl...|      

### Spalten auswählen (`select`)

Mit `.select()` kannst du gezielt bestimmte Spalten aus dem DataFrame auswählen.

**Merke:**  
> `.select()` **verändert** das ursprüngliche DataFrame **nicht**.  
> Es erzeugt eine **neue** Version.

In [19]:
df.select("name", "city", "job_title").show()

+-------+----------+-------------------+
|   name|      city|          job_title|
+-------+----------+-------------------+
|   Anna|    Berlin|       Data Analyst|
|    Ben|   Hamburg|  Software Engineer|
|  Clara|    Munich|Marketing Assistant|
|  David|   Cologne|    Project Manager|
|    Eva| Stuttgart|     Data Scientist|
|  Felix|    Berlin|    DevOps Engineer|
|   Gina|   Hamburg|         HR Manager|
| Hannah| Frankfurt|        UI Designer|
|   Igor|    Munich|      Sales Manager|
|  Julia|Düsseldorf|    Content Creator|
|   Karl|    Berlin|                CTO|
|   Lina| Stuttgart|         Accountant|
|    Max|   Cologne|   Network Engineer|
|   Nina|   Hamburg|      Product Owner|
| Oliver|    Berlin|  Backend Developer|
|  Paula|    Munich|   Product Designer|
|Quentin| Frankfurt|         Consultant|
|   Rita|Düsseldorf|   Junior Developer|
| Stefan| Stuttgart|         IT Support|
|   Tina|   Cologne|      Data Engineer|
+-------+----------+-------------------+



### Zeilen filtern (`filter`, `where`)

Mit `.filter()` oder `.where()` kannst du Zeilen nach Bedingungen auswählen.

In [20]:
df.filter(df.age > 30).show()

+-------+---+---------+--------------------+-----------------+------+
|   name|age|     city|               email|        job_title|salary|
+-------+---+---------+--------------------+-----------------+------+
|    Ben| 35|  Hamburg|ben.schmidt@examp...|Software Engineer| 74000|
|  David| 40|  Cologne|david.schneider@e...|  Project Manager| 83000|
|    Eva| 31|Stuttgart|eva.huber@example...|   Data Scientist| 68000|
|   Gina| 45|  Hamburg|gina.fischer@exam...|       HR Manager| 69000|
|   Igor| 38|   Munich|igor.keller@examp...|    Sales Manager| 75000|
|   Karl| 50|   Berlin|karl.bauer@exampl...|              CTO|120000|
|   Lina| 33|Stuttgart|lina.maier@exampl...|       Accountant| 58000|
|    Max| 41|  Cologne|max.frank@example...| Network Engineer| 71000|
|  Paula| 36|   Munich|paula.hartmann@ex...| Product Designer| 65000|
|Quentin| 43|Frankfurt|quentin.schulz@ex...|       Consultant| 80000|
| Stefan| 39|Stuttgart|stefan.becker@exa...|       IT Support| 56000|
|   Tina| 32|  Colog

In [21]:
df.filter(df.salary > 80000).show()

+-----+---+-------+--------------------+---------------+------+
| name|age|   city|               email|      job_title|salary|
+-----+---+-------+--------------------+---------------+------+
|David| 40|Cologne|david.schneider@e...|Project Manager| 83000|
| Karl| 50| Berlin|karl.bauer@exampl...|            CTO|120000|
+-----+---+-------+--------------------+---------------+------+



In [22]:
df.where(df.age > 30).show()

+-------+---+---------+--------------------+-----------------+------+
|   name|age|     city|               email|        job_title|salary|
+-------+---+---------+--------------------+-----------------+------+
|    Ben| 35|  Hamburg|ben.schmidt@examp...|Software Engineer| 74000|
|  David| 40|  Cologne|david.schneider@e...|  Project Manager| 83000|
|    Eva| 31|Stuttgart|eva.huber@example...|   Data Scientist| 68000|
|   Gina| 45|  Hamburg|gina.fischer@exam...|       HR Manager| 69000|
|   Igor| 38|   Munich|igor.keller@examp...|    Sales Manager| 75000|
|   Karl| 50|   Berlin|karl.bauer@exampl...|              CTO|120000|
|   Lina| 33|Stuttgart|lina.maier@exampl...|       Accountant| 58000|
|    Max| 41|  Cologne|max.frank@example...| Network Engineer| 71000|
|  Paula| 36|   Munich|paula.hartmann@ex...| Product Designer| 65000|
|Quentin| 43|Frankfurt|quentin.schulz@ex...|       Consultant| 80000|
| Stefan| 39|Stuttgart|stefan.becker@exa...|       IT Support| 56000|
|   Tina| 32|  Colog

**Hinweis:**  
> **`filter`** und **`where`** sind **identisch** – es ist reine Geschmackssache.

### Neue Spalten erstellen (`withColumn`)

In [23]:
from pyspark.sql.functions import col

df = df.withColumn("age_plus_5", col("age") + 5)
df.show()

+-------+---+----------+--------------------+-------------------+------+----------+
|   name|age|      city|               email|          job_title|salary|age_plus_5|
+-------+---+----------+--------------------+-------------------+------+----------+
|   Anna| 28|    Berlin|anna.mueller@exam...|       Data Analyst| 52000|      33.0|
|    Ben| 35|   Hamburg|ben.schmidt@examp...|  Software Engineer| 74000|      40.0|
|  Clara| 22|    Munich|clara.klein@examp...|Marketing Assistant| 42000|      27.0|
|  David| 40|   Cologne|david.schneider@e...|    Project Manager| 83000|      45.0|
|    Eva| 31| Stuttgart|eva.huber@example...|     Data Scientist| 68000|      36.0|
|  Felix| 29|    Berlin|felix.wagner@exam...|    DevOps Engineer|  NULL|      34.0|
|   Gina| 45|   Hamburg|gina.fischer@exam...|         HR Manager| 69000|      50.0|
| Hannah| 26| Frankfurt|hannah.koch@examp...|        UI Designer| 54000|      31.0|
|   Igor| 38|    Munich|igor.keller@examp...|      Sales Manager| 75000|    

**Wichtig:**  
> Jede `.withColumn()`-Operation erstellt **intern ein neues DataFrame** — Spark verändert nie das Originalobjekt direkt.

### Bedingte Logik (`when`, `otherwise`)

Mit `when` und `otherwise` kannst du **Bedingungen** einbauen, ähnlich wie `if-else`.

In [33]:
from pyspark.sql.functions import when

df = df.withColumn(
    "income_category",
    when(col("salary") >= 80000, "Hochverdiener")
    .when((col("salary") >= 43000) & (col("salary") < 80000), "Normalverdiener")
    .otherwise("Geringverdiener")
)
df.show()

+-------+---+----------+--------------------+-------------------+------+----------+---------------+
|   name|age|      city|               email|          job_title|salary|age_plus_5|income_category|
+-------+---+----------+--------------------+-------------------+------+----------+---------------+
|   Anna| 28|    Berlin|anna.mueller@exam...|       Data Analyst| 52000|      33.0|Normalverdiener|
|    Ben| 35|   Hamburg|ben.schmidt@examp...|  Software Engineer| 74000|      40.0|Normalverdiener|
|  Clara| 22|    Munich|clara.klein@examp...|Marketing Assistant| 42000|      27.0|Geringverdiener|
|  David| 40|   Cologne|david.schneider@e...|    Project Manager| 83000|      45.0|  Hochverdiener|
|    Eva| 31| Stuttgart|eva.huber@example...|     Data Scientist| 68000|      36.0|Normalverdiener|
|  Felix| 29|    Berlin|felix.wagner@exam...|    DevOps Engineer|  NULL|      34.0|Geringverdiener|
|   Gina| 45|   Hamburg|gina.fischer@exam...|         HR Manager| 69000|      50.0|Normalverdiener|


**Merke:**  
> Viele verschachtelte `when`-Bedingungen können unübersichtlich werden → sauber strukturieren!

### Umgang mit fehlenden Werten (`fillna`, `dropna`)

In [30]:
# Fehlende Werte füllen (`fillna`):
# Ersetzt fehlende Gehälter durch 50000
df_filled = df.fillna({"salary": 50000})
df_filled.show()

+-------+---+----------+--------------------+-------------------+------+----------+---------------+
|   name|age|      city|               email|          job_title|salary|age_plus_5|income_category|
+-------+---+----------+--------------------+-------------------+------+----------+---------------+
|   Anna| 28|    Berlin|anna.mueller@exam...|       Data Analyst| 52000|      33.0|Normalverdiener|
|    Ben| 35|   Hamburg|ben.schmidt@examp...|  Software Engineer| 74000|      40.0|Normalverdiener|
|  Clara| 22|    Munich|clara.klein@examp...|Marketing Assistant| 42000|      27.0|Geringverdiener|
|  David| 40|   Cologne|david.schneider@e...|    Project Manager| 83000|      45.0|  Hochverdiener|
|    Eva| 31| Stuttgart|eva.huber@example...|     Data Scientist| 68000|      36.0|Normalverdiener|
|  Felix| 29|    Berlin|felix.wagner@exam...|    DevOps Engineer| 50000|      34.0|Geringverdiener|
|   Gina| 45|   Hamburg|gina.fischer@exam...|         HR Manager| 69000|      50.0|Normalverdiener|


In [29]:
# Zeilen mit fehlenden Werten löschen (`dropna`):
# Entfernt alle Zeilen, die mindestens einen `null`-Wert enthalten.
df_clean = df.dropna()
df_clean.show()

+-------+---+----------+--------------------+-------------------+------+----------+---------------+
|   name|age|      city|               email|          job_title|salary|age_plus_5|income_category|
+-------+---+----------+--------------------+-------------------+------+----------+---------------+
|   Anna| 28|    Berlin|anna.mueller@exam...|       Data Analyst| 52000|      33.0|Normalverdiener|
|    Ben| 35|   Hamburg|ben.schmidt@examp...|  Software Engineer| 74000|      40.0|Normalverdiener|
|  Clara| 22|    Munich|clara.klein@examp...|Marketing Assistant| 42000|      27.0|Geringverdiener|
|  David| 40|   Cologne|david.schneider@e...|    Project Manager| 83000|      45.0|  Hochverdiener|
|    Eva| 31| Stuttgart|eva.huber@example...|     Data Scientist| 68000|      36.0|Normalverdiener|
|   Gina| 45|   Hamburg|gina.fischer@exam...|         HR Manager| 69000|      50.0|Normalverdiener|
| Hannah| 26| Frankfurt|hannah.koch@examp...|        UI Designer| 54000|      31.0|Normalverdiener|


### Gruppieren und Aggregieren (`groupBy` + `agg`)

Mit `.groupBy()` kannst du dein DataFrame nach einer oder mehreren Spalten **gruppieren**.  
Mit `.agg()` kannst du dann **Aggregationfunktionen** auf jede Gruppe anwenden.

In [35]:
from pyspark.sql import functions as F

#Durchschnitt berechnen
df.groupBy("city").agg(F.avg("salary")).show()

+----------+------------------+
|      city|       avg(salary)|
+----------+------------------+
| Frankfurt|           67000.0|
|    Berlin| 79666.66666666667|
|Düsseldorf|           44000.0|
|   Hamburg|           71500.0|
| Stuttgart|60666.666666666664|
|   Cologne| 75666.66666666667|
|    Munich|60666.666666666664|
+----------+------------------+



In [36]:
#Zählt die Anzahl der Einträge
df.groupBy("city").count().show()

+----------+-----+
|      city|count|
+----------+-----+
| Frankfurt|    2|
|    Berlin|    4|
|Düsseldorf|    2|
|   Hamburg|    3|
| Stuttgart|    3|
|   Cologne|    3|
|    Munich|    3|
+----------+-----+



In [37]:
#Summiert Werte einer Spalte
df.groupBy("city").agg(F.sum("salary")).show()

+----------+-----------+
|      city|sum(salary)|
+----------+-----------+
| Frankfurt|   134000.0|
|    Berlin|   239000.0|
|Düsseldorf|    88000.0|
|   Hamburg|   143000.0|
| Stuttgart|   182000.0|
|   Cologne|   227000.0|
|    Munich|   182000.0|
+----------+-----------+



**Wichtig zu wissen:**  
> `.groupBy()` alleine macht noch nichts. Erst `.agg()`, `.count()`, `.sum()` oder `.avg()` lösen die echte Berechnung aus.

### Sortieren und Reihenfolge ändern (`orderBy`, `sort`)

Mit `.orderBy()` oder `.sort()` kannst du dein DataFrame sortieren.

In [39]:
#Nach Gehalt absteigend sortieren
df.orderBy(F.desc("salary")).show()

+-------+---+----------+--------------------+-------------------+------+----------+---------------+
|   name|age|      city|               email|          job_title|salary|age_plus_5|income_category|
+-------+---+----------+--------------------+-------------------+------+----------+---------------+
|  David| 40|   Cologne|david.schneider@e...|    Project Manager| 83000|      45.0|  Hochverdiener|
|Quentin| 43| Frankfurt|quentin.schulz@ex...|         Consultant| 80000|      48.0|  Hochverdiener|
|   Igor| 38|    Munich|igor.keller@examp...|      Sales Manager| 75000|      43.0|Normalverdiener|
|    Ben| 35|   Hamburg|ben.schmidt@examp...|  Software Engineer| 74000|      40.0|Normalverdiener|
|   Tina| 32|   Cologne|tina.kraus@exampl...|      Data Engineer| 73000|      37.0|Normalverdiener|
|    Max| 41|   Cologne|max.frank@example...|   Network Engineer| 71000|      46.0|Normalverdiener|
|   Gina| 45|   Hamburg|gina.fischer@exam...|         HR Manager| 69000|      50.0|Normalverdiener|


In [38]:
# Oder aufsteigend (Standard)
df.orderBy("age").show()

+-------+---+----------+--------------------+-------------------+------+----------+---------------+
|   name|age|      city|               email|          job_title|salary|age_plus_5|income_category|
+-------+---+----------+--------------------+-------------------+------+----------+---------------+
|  Clara| 22|    Munich|clara.klein@examp...|Marketing Assistant| 42000|      27.0|Geringverdiener|
|  Julia| 24|Düsseldorf|julia.schmitt@exa...|    Content Creator| 41000|      29.0|Geringverdiener|
|   Rita| 25|Düsseldorf|rita.lang@example...|   Junior Developer| 47000|      30.0|Normalverdiener|
| Hannah| 26| Frankfurt|hannah.koch@examp...|        UI Designer| 54000|      31.0|Normalverdiener|
| Oliver| 27|    Berlin|oliver.weber@exam...|  Backend Developer| 67000|      32.0|Normalverdiener|
|   Anna| 28|    Berlin|anna.mueller@exam...|       Data Analyst| 52000|      33.0|Normalverdiener|
|  Felix| 29|    Berlin|felix.wagner@exam...|    DevOps Engineer|  NULL|      34.0|Geringverdiener|


In [ ]:
## 🔃 Sortieren und Reihenfolge ändern (`orderBy`, `sort`)

Mit `.orderBy()` oder `.sort()` kannst du dein DataFrame sortieren.

**Beispiel: Nach Gehalt absteigend sortieren:**

```python
df.orderBy(F.desc("salary")).show()
```

**Oder aufsteigend (Standard):**

```python
df.orderBy("age").show()

**Hinweis:**  
> `orderBy` und `sort` sind **gleichwertig**.  
> Bei riesigen DataFrames kann Sortieren teuer werden → vorsichtig einsetzen!

### Window Functions

Window Functions erlauben dir, über eine **Teilmenge** deiner Daten zu arbeiten, **ohne** sie vollständig zu aggregieren.

In [40]:
from pyspark.sql.window import Window

window_spec = Window.partitionBy("city").orderBy(F.desc("salary"))

df = df.withColumn("rank_in_city", F.rank().over(window_spec))
df.show()

+-------+---+----------+--------------------+-------------------+------+----------+---------------+------------+
|   name|age|      city|               email|          job_title|salary|age_plus_5|income_category|rank_in_city|
+-------+---+----------+--------------------+-------------------+------+----------+---------------+------------+
| Oliver| 27|    Berlin|oliver.weber@exam...|  Backend Developer| 67000|      32.0|Normalverdiener|           1|
|   Anna| 28|    Berlin|anna.mueller@exam...|       Data Analyst| 52000|      33.0|Normalverdiener|           2|
|   Karl| 50|    Berlin|karl.bauer@exampl...|                CTO|120000|      55.0|  Hochverdiener|           3|
|  Felix| 29|    Berlin|felix.wagner@exam...|    DevOps Engineer|  NULL|      34.0|Geringverdiener|           4|
|  David| 40|   Cologne|david.schneider@e...|    Project Manager| 83000|      45.0|  Hochverdiener|           1|
|   Tina| 32|   Cologne|tina.kraus@exampl...|      Data Engineer| 73000|      37.0|Normalverdien

25/04/29 08:00:42 ERROR Inbox: Ignoring error
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.SparkThreadUtils$.awaitResult(SparkThreadUtils.scala:56)
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:310)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRefByURI(RpcEnv.scala:102)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRef(RpcEnv.scala:110)
	at org.apache.spark.util.RpcUtils$.makeDriverRef(RpcUtils.scala:36)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.driverEndpoint$lzycompute(BlockManagerMasterEndpoint.scala:124)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.org$apache$spark$storage$BlockManagerMasterEndpoint$$driverEndpoint(BlockManagerMasterEndpoint.scala:123)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.isExecutorAlive$lzycompute$1(BlockManagerMasterEndpoint.scala:688)
	at org.apache.spark.storage.BlockManagerMasterE

### Lesen von Azure Datenbank

In [ ]:
from pyspark.sql import SparkSession
from dotenv import load_dotenv
import os

spark = SparkSession.builder \
    .appName("Localspark") \
    .master("local[*]") \
    .config("spark.jars.packages",
        "com.microsoft.sqlserver:mssql-jdbc:12.6.1.jre11") \
    .getOrCreate()
spark.sparkContext.setLogLevel("ERROR")

# Hour 1: Introduction to PySpark

- What is Apache Spark?
- Differences between PySpark and Pandas
- Spark architecture overview (driver, executor, cluster manager)
- Setting up Spark locally using Docker or pip
- Creating a SparkSession

In [ ]:
# Already included, reinforce
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("IntroSession").getOrCreate()
spark.version

# Hour 2: DataFrame Operations

- Creating DataFrames
- Selecting, filtering, adding columns
- Aggregation and grouping
- Hands-on: Word count and basic operations

In [ ]:
# Exercise: Create a DataFrame and perform groupBy aggregation
data = [("apple", 2), ("banana", 3), ("apple", 1)]
df = spark.createDataFrame(data, ["fruit", "count"])
df.groupBy("fruit").sum("count").show()

# Hour 3: Working with External Data

- Reading CSV, JSON, Parquet
- Writing data to storage
- Connecting to databases using JDBC (already introduced)
- Data cleaning and null handling

In [ ]:
# Example: Read a CSV file
csv_df = spark.read.option("header", True).csv("sample_data.csv")
csv_df.show()

# Hour 4: Advanced Transformations & Spark SQL

- Using UDFs
- Explode, arrays, maps
- Date/timestamp operations
- Writing and running SQL queries

In [ ]:
# Example: Create temp view and use SQL
csv_df.createOrReplaceTempView("data")
sql_result = spark.sql("SELECT fruit, COUNT(*) FROM data GROUP BY fruit")
sql_result.show()

# Hour 5: Optimization and Performance

- Spark execution plan with `explain()`
- Caching and persistence
- Partitioning strategies
- Join types and broadcast joins

In [ ]:
# Example: Show query plan
df.explain()

# Hour 6: Mini Project + Recap

- Load a dataset
- Perform transformations
- Apply SQL or aggregations
- Write results to file or database
- Wrap-up discussion and review

In [ ]:
# Placeholder for project code
data = [("2023-01-01", 100), ("2023-01-02", 120)]
df = spark.createDataFrame(data, ["date", "sales"])
df.show()